In [1]:
import numpy as np
import pandas as pd

In [5]:
! pip install pillow


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import os
from PIL import Image
import xml.etree.ElementTree as ET
import numpy as np

# 이미지와 XML 파일이 있는 폴더 경로를 설정합니다.
image_folder = 'images'
xml_folder = 'annotations'

# 이미지 크기를 설정합니다. (YOLO 입력 이미지 크기)
input_size = (416, 416)

# YOLO 모델에 입력으로 사용할 데이터셋을 저장할 리스트를 생성합니다.
dataset = []

# annotations 폴더 안에 있는 모든 XML 파일을 순회합니다.
for filename in os.listdir(xml_folder):
    if filename.endswith('.xml'):
        xml_path = os.path.join(xml_folder, filename)
        
        # XML 파일을 파싱하여 바운딩 박스 정보를 추출합니다.
        tree = ET.parse(xml_path)
        root = tree.getroot()
        
        # 바운딩 박스 요소를 찾고, 요소가 존재하지 않는 경우 해당 이미지를 건너뜁니다.
        xmin_element = root.find('.//xmin')
        ymin_element = root.find('.//ymin')
        xmax_element = root.find('.//xmax')
        ymax_element = root.find('.//ymax')
        if xmin_element is None or ymin_element is None or xmax_element is None or ymax_element is None:
            print(f"WARNING: {filename} - Bounding box information is missing. Skipping this image.")
            continue
        
        xmin = int(xmin_element.text)
        ymin = int(ymin_element.text)
        xmax = int(xmax_element.text)
        ymax = int(ymax_element.text)
        
        # 이미지 파일 경로를 생성합니다.
        image_filename = os.path.splitext(filename)[0] + '.png'
        image_path = os.path.join(image_folder, image_filename)
        
        # 이미지 로드
        image = Image.open(image_path)
        
        # 이미지 크기 조정
        image = image.resize(input_size)
        
        # 이미지를 numpy 배열로 변환하고 픽셀 값을 0과 1 사이로 스케일링
        image_array = np.array(image) / 255.0
        
        # 바운딩 박스 좌표를 정규화하여 YOLO 형식으로 변환
        x_center = (xmin + xmax) / 2 / image.width
        y_center = (ymin + ymax) / 2 / image.height
        width = (xmax - xmin) / image.width
        height = (ymax - ymin) / image.height
        
        # YOLO 모델에 입력으로 사용할 형식으로 데이터를 저장
        dataset.append([image_array, x_center, y_center, width, height])

# 데이터셋을 저장하고 나중에 훈련에 사용할 수 있도록 준비합니다.
# dataset를 훈련에 적합한 형태로 변환하고, DataLoader 등을 이용하여 사용할 수 있습니다.


In [16]:
! python -m pip install yolov5


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
!pip install torch torchvision


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
! python -m pip install --upgrade pip

     ---------------------------------------- 2.1/2.1 MB 1.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [19]:
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import functional as F
from pathlib import Path
from tqdm.notebook import tqdm

# 데이터셋 로드 (전처리된 데이터셋 사용)
# dataset = YourCustomDataset(...)  # 전처리된 데이터셋을 로드하는 방법에 따라 코드를 수정

# 데이터 로더 설정
batch_size = 8
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# YOLOv5 모델 생성
from yolov5.models import YOLOv5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = YOLOv5(num_classes=...).to(device)  # 클래스 개수에 따라 변경

# 옵티마이저 설정
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 손실 함수 설정 (YoloLoss 사용 예시)
from yolov5.loss import YoloLoss

criterion = YoloLoss(...).to(device)  # 필요한 인자들에 따라 설정 (예: anchors, num_classes 등)

# 학습 설정
epochs = 10

# 훈련 시작
model.train()
for epoch in range(epochs):
    total_loss = 0.0
    for batch in tqdm(dataloader, desc=f'Epoch {epoch + 1}/{epochs}'):
        imgs, targets = batch

        imgs = [F.to_tensor(img).to(device) for img in imgs]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Forward
        outputs = model(imgs)

        # Loss 계산
        loss = criterion(outputs, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader)}")

# 훈련된 모델 저장
model_save_path = 'path_to_save_model.pth'
torch.save(model.state_dict(), model_save_path)


ModuleNotFoundError: No module named 'torch'